## Graph Neural Networks with Pytorch
## Target: RGCN: Modeling Relational Data with Graph Convolutional Networks
- Original Paper: https://arxiv.org/abs/1703.06103
- Original Code: [Example1](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/rgcn.py), [Example2](https://github.com/rusty1s/pytorch_geometric/blob/master/examples/rgcn_link_pred.py)

In [1]:
import os, sys

from tqdm import tqdm

import torch
import torch.nn.functional as F
from torch.nn import Parameter

from torch_geometric.nn import GAE, RGCNConv

try:
    from torch_geometric.datasets import RelLinkPredDataset
except ImportError:
    from fb_dataset import RelLinkPredDataset

본 노트북에서는 **RGCN**이란 알고리즘에 대해 알아보고, 간단히 코드를 실행시켜 볼 것이다.  

Knowledge bases에서 missing information에 대해 예측하는 것은 SRL(Statistical Relational Learning)에서 굉장히 중요한 문제이다. 이 때 knowledge bases는 (subject, predicate, object)과 같이 collections of triples의 정보를 저장한다.  

예를 들어, (youyoung, was_in_the, room)에서 subject인 youyoung과 object인 room은 entity이고, predicate인 was_in_the는 relation이 된다. 그리고 이 entity는 특정 type을 갖게 된다.  

위와 같은 형태의 관계를 그래프로 나타낸 것이 **Relational Graph**이고, node는 entity, edge는 relation이 된다.  

**RGCN**의 layer update 식은 아래와 같다.  

$$ h_i^{l+1} = \sigma( \Sigma_{r \in \mathcal{R}} \Sigma_{j \in \mathcal{N}_i^r} \frac{1}{c_{i, r}} W_r^l h_j^l + W_o^l h_i^l) $$  

이 때 $c_{i, r}$ 의 경우 $\vert \mathcal{N}_i^r \vert$ 와 같이 상수로 정할 수도 있고, 학습 가능한 attention score로 설정할 수도 있다. 위 식은 실제 구현될 때는 행렬식으로 변경해 주어야 한다.  

일단 학습 데이터를 불러와보자.

In [2]:
path = os.path.join(os.getcwd(), '..', 'data', 'RLPD')
dataset = RelLinkPredDataset(path, 'FB15k-237')
data = dataset[0]

print(data)
print(f"\n Num Relations: {dataset.num_relations}")

Data(edge_index=[2, 544230], edge_type=[544230], test_edge_index=[2, 20466], test_edge_type=[20466], train_edge_index=[2, 272115], train_edge_type=[272115], valid_edge_index=[2, 17535], valid_edge_type=[17535])

 Num Relations: 474


**RGCN**는 목표 task에 따라 구현 방식이 달라진다. Entity Classification을 기준으로 코드가 짜여져 있는 것이고, Link Prediction을 목표로 할 경우 RGCN Encoder + DistMult Decoder의 형식을 갖추게 된다. 본 노트북에서는 후자에 대해 살펴본다.  

모델의 특성 상 relation의 종류가 굉장히 많은 데이터에 적용할 경우 파라미터의 수가 크게 증가하고 이러한 특징은 과적합으로 이어질 가능성이 있다. 이에 대응하기 위해 **RGCN**은 2가지 규제 방안을 제시하고 있다. 적용해본 후에 더 나은 선택지를 고르면 될 것이다. (굉장히 흥미로운 방법이다.)  

1번째 방법은 `Basis Decomposition`이다.  

$$ W_r^l = \Sigma_{b=1}^B a_{rb}^l V_b^l $$  

$V_b^l$ 은 relation에 의존하지 않고 오직 $a$ 만 relation에 따라 늘어나게 된다. 즉 이 방법은 **Weight Sharing**을 의미한다. 전체적으로 $V$ 를 학습하는데에 초점을 두고, relation에 따라 $a$ 로 조절하게 되는 것이다.  

2번째 방법은 **Block-diagonal Matrices**를 이용하는 것이다.  

$$ W_r^l = \bigoplus_{b=1}^B Q_{br}^l $$  

이 때 $Q$ 는 block-diagonal 행렬을 의미하게 된다. $W$ 의 shape이 $(d^{l+1}, d^l)$ 이라고 할 때, $Q$ 의 shape은 (d^{l+1}/B, d^l/B)가 된다.  

$$
\begin{bmatrix}
Q_{1r}^l & & \\
& \ddots & \\
& & Q_{Br}^l
\end{bmatrix}
$$

이렇게 되면 Sparsity Constraint의 효과를 갖게 되면서 Weigth Matrix를 regularize하게 된다. 다만 근접한 차원만이 $W$ 와 interact할 수 있다는 한계를 지니게 된다.  

`RGCONConv`에 대해서는 [공식 문서](https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.RGCNConv)에서 자세한 설명을 확인할 수 있다.  

아래 코드를 보면 argument는 매우 직관적임을 알 수 있다.

In [3]:
conv = RGCNConv(
    in_channels=data.num_nodes, out_channels=16, num_relations=dataset.num_relations,
    num_bases=30, num_blocks=None, aggr='mean')

In [4]:
class RGCNEncoder(torch.nn.Module):
    def __init__(self, num_nodes, hidden_channels, num_relations):
        super().__init__()
        self.node_emb = Parameter(torch.Tensor(num_nodes, hidden_channels))
        self.conv1 = RGCNConv(hidden_channels, hidden_channels, num_relations, num_blocks=5)
        self.conv2 = RGCNConv(hidden_channels, hidden_channels, num_relations, num_blocks=5)
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.node_emb)
        self.conv1.reset_parameters()
        self.conv2.reset_parameters()

    def forward(self, edge_index, edge_type):
        x = self.node_emb
        x = self.conv1(x, edge_index, edge_type).relu_()
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index, edge_type)
        return x

In [5]:
class DistMultDecoder(torch.nn.Module):
    def __init__(self, num_relations, hidden_channels):
        super().__init__()
        self.rel_emb = Parameter(torch.Tensor(num_relations, hidden_channels))
        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.rel_emb)

    def forward(self, z, edge_index, edge_type):
        z_src, z_dst = z[edge_index[0]], z[edge_index[1]]
        rel = self.rel_emb[edge_type]
        return torch.sum(z_src * rel * z_dst, dim=1)

In [6]:
model = GAE(
    RGCNEncoder(data.num_nodes, hidden_channels=500, num_relations=dataset.num_relations),
    DistMultDecoder(dataset.num_relations // 2, hidden_channels=500),
)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [7]:
print(model)

GAE(
  (encoder): RGCNEncoder(
    (conv1): RGCNConv(500, 500, num_relations=474)
    (conv2): RGCNConv(500, 500, num_relations=474)
  )
  (decoder): DistMultDecoder()
)


In [8]:
def negative_sampling(edge_index, num_nodes):
    # Sample edges by corrupting either the subject or the object of each edge.
    mask_1 = torch.rand(edge_index.size(1)) < 0.5
    mask_2 = mask_1

    neg_edge_index = edge_index.clone()
    neg_edge_index[0, mask_1] = torch.randint(num_nodes, (mask_1.sum(), ))
    neg_edge_index[1, mask_2] = torch.randint(num_nodes, (mask_2.sum(), ))
    return neg_edge_index

In [9]:
def train():
    model.train()
    optimizer.zero_grad()

    z = model.encode(data.edge_index, data.edge_type)

    pos_out = model.decode(z, data.train_edge_index, data.train_edge_type)

    neg_edge_index = negative_sampling(data.train_edge_index, data.num_nodes)
    neg_out = model.decode(z, neg_edge_index, data.train_edge_type)

    out = torch.cat([pos_out, neg_out])
    gt = torch.cat([torch.ones_like(pos_out), torch.zeros_like(neg_out)])
    cross_entropy_loss = F.binary_cross_entropy_with_logits(out, gt)
    reg_loss = z.pow(2).mean() + model.decoder.rel_emb.pow(2).mean()
    loss = cross_entropy_loss + 1e-2 * reg_loss

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.)
    optimizer.step()

    return float(loss)


@torch.no_grad()
def test():
    model.eval()
    z = model.encode(data.edge_index, data.edge_type)

    valid_mrr = compute_mrr(z, data.valid_edge_index, data.valid_edge_type)
    test_mrr = compute_mrr(z, data.test_edge_index, data.test_edge_type)

    return valid_mrr, test_mrr


@torch.no_grad()
def compute_mrr(z, edge_index, edge_type):
    ranks = []
    for i in tqdm(range(edge_type.numel())):
        (src, dst), rel = edge_index[:, i], edge_type[i]

        # Try all nodes as tails, but delete true triplets:
        tail_mask = torch.ones(data.num_nodes, dtype=torch.bool)
        for (heads, tails), types in [
            (data.train_edge_index, data.train_edge_type),
            (data.valid_edge_index, data.valid_edge_type),
            (data.test_edge_index, data.test_edge_type),
        ]:
            tail_mask[tails[(heads == src) & (types == rel)]] = False

        tail = torch.arange(data.num_nodes)[tail_mask]
        tail = torch.cat([torch.tensor([dst]), tail])
        head = torch.full_like(tail, fill_value=src)
        eval_edge_index = torch.stack([head, tail], dim=0)
        eval_edge_type = torch.full_like(tail, fill_value=rel)

        out = model.decode(z, eval_edge_index, eval_edge_type)
        perm = out.argsort(descending=True)
        rank = int((perm == 0).nonzero(as_tuple=False).view(-1)[0])
        ranks.append(rank + 1)

        # Try all nodes as heads, but delete true triplets:
        head_mask = torch.ones(data.num_nodes, dtype=torch.bool)
        for (heads, tails), types in [
            (data.train_edge_index, data.train_edge_type),
            (data.valid_edge_index, data.valid_edge_type),
            (data.test_edge_index, data.test_edge_type),
        ]:
            head_mask[heads[(tails == dst) & (types == rel)]] = False

        head = torch.arange(data.num_nodes)[head_mask]
        head = torch.cat([torch.tensor([src]), head])
        tail = torch.full_like(head, fill_value=dst)
        eval_edge_index = torch.stack([head, tail], dim=0)
        eval_edge_type = torch.full_like(head, fill_value=rel)

        out = model.decode(z, eval_edge_index, eval_edge_type)
        perm = out.argsort(descending=True)
        rank = int((perm == 0).nonzero(as_tuple=False).view(-1)[0])
        ranks.append(rank + 1)

    return (1. / torch.tensor(ranks, dtype=torch.float)).mean()

In [ ]:
# 경고: 정말 오래 걸린다.
for epoch in range(1, 1001):
    loss = train()
    print(f'Epoch: {epoch:05d}, Loss: {loss:.4f}')
    if (epoch % 500) == 0:
        valid_mrr, test_mrr = test()
        print(f'Val MRR: {valid_mrr:.4f}, Test MRR: {test_mrr:.4f}')